In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
nasnetlarge = \
    model_development.nasnetlarge(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_classification_model(\
    training_phase = 1, model = nasnetlarge, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'NasNetLarge', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

343610240/343610240 [==============================] - 15s 0us/step
Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.7191 - accuracy: 0.6612 - precision: 0.6787 - recall: 0.4814
Epoch 1: val_accuracy improved from -inf to 0.70448, saving model to ../models/weights\NasNetLarge_1.00.00_fs.hdf5
119/119 [==============================] - 77s 489ms/step - loss: 0.7191 - accuracy: 0.6612 - precision: 0.6787 - recall: 0.4814 - val_loss: 0.5717 - val_accuracy: 0.7045 - val_precision: 0.7928 - val_recall: 0.5366
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5321 - accuracy: 0.7335 - precision: 0.7622 - recall: 0.6000
Epoch 2: val_accuracy did not improve from 0.70448
119/119 [==============================] - 51s 432ms/step - loss: 0.5321 - accuracy: 0.7335 - precision: 0.7622 - recall: 0.6000 - val_loss: 0.5502 - val_accuracy: 0.6955 - val_precision: 0.6782 - val_recall: 0.7195
Epoch 3/200
119/119 [==============================] - ETA: 0s - l

In [6]:
##
#Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/NasNetLarge_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'NasNetLarge', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 8.7196 - accuracy: 0.6269 - precision: 0.5825 - recall: 0.6279
Epoch 1: val_accuracy improved from -inf to 0.62985, saving model to ../models/weights\NasNetLarge_1.00.00_ss.hdf5
119/119 [==============================] - 169s 1s/step - loss: 8.7196 - accuracy: 0.6269 - precision: 0.5825 - recall: 0.6279 - val_loss: 0.6602 - val_accuracy: 0.6299 - val_precision: 0.9545 - val_recall: 0.2561
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5535 - accuracy: 0.7240 - precision: 0.8363 - recall: 0.4872
Epoch 2: val_accuracy improved from 0.62985 to 0.68657, saving model to ../models/weights\NasNetLarge_1.00.00_ss.hdf5
119/119 [==============================] - 144s 1s/step - loss: 0.5535 - accuracy: 0.7240 - precision: 0.8363 - recall: 0.4872 - val_loss: 0.5285 - val_accuracy: 0.6866 - val_precision: 0.9041 - val_recall: 0.4024
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 

In [ ]:
##
# Clear memory after model training:
# 
model_development.clear_memory()